Name: Haw Xiao Ying <br>Student ID: 29797918 <br>Email: xhaw0001@student.monash.edu

# Assignment Part B

## Task 1(d)

In [ ]:
import pygeohash as pgh

def near_location(lat1, lon1, lat2, lon2):
    location1 = pgh.encode(lat1, lon1, precision=3)
    location2 = pgh.encode(lat2, lon2, precision=3)
    if location1 == location2:
        return True
    else:
        return False
    
def get_hash_precision_5(lat1, lon1):
    location1 = pgh.encode(lat1, lon1, precision=5)
    return(location1)

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'

import sys
import time
import json
import random
import pandas as pd
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
                
                
def sendDataToDB(iter):
    
    climate = []
    aqua = []
    terra = []
    
    for record in iter:
        key = record[0]
        if key == "producer_1":
            climate.append(json.loads(record[1]))
        elif key == "producer_2":
            aqua.append(json.loads(record[1]))
        else:
            terra.append(json.loads(record[1]))
            
    
    # If there is no climate data, then just end it
    if len(climate) != 0:
        
        # Check if hotspot is near to climate
        #  - Not close: ignore the hotspot data
        #  - Close: store it into near_hotspot list for further usage
        near_hotspot = []
        for i in climate:
            for j in aqua:
                if (near_location(i['latitude'], i['longitude'], j['latitude'], j['longitude'])):
                    near_hotspot.append(j)
            for j in terra:
                if (near_location(i['latitude'], i['longitude'], j['latitude'], j['longitude'])):
                    near_hotspot.append(j)
        
        # If there are no near hotspots, then just store climate data only
        if len(near_hotspot) != 0:
            
            # Get the hash of all hotspot
            hotspot_hash = []
            for location in near_hotspot:
                hotspot_hash.append(get_hash_precision_5(location['latitude'], location['longitude']))
            
            # Check if there is same hash indicating same locations
            temp = []
            same_hotspot = []
            for h in range(len(hotspot_hash)):
                if hotspot_hash[h] not in temp:
                    temp.append(hotspot_hash[h])
                else:
                    same_hotspot.append(h)
                
            # If there are hotspots with same location, merge them into 1
            if len(same_hotspot) != 0:
                temp = []
                conf = []
                lat = []
                lon = []
                same_hotspot = same_hotspot[::-1]
                dic['time'] = near_hotspot[same_hotspot[0]]['time']
                for index in same_hotspot:
                    temp.append(near_hotspot[index]['surface_temperature_celcius'])
                    conf.append(near_hotspot[index]['confidence'])
                    lat.append(near_hotspot[index]['latitude'])
                    lon.append(near_hotspot[index]['longitude'])
                    producer = near_hotspot[index]['producer']
                    near_hotspot.remove(near_hotspot[index])
                avg_stc = round(sum(temp)/len(temp))
                avg_c = round(sum(conf)/len(conf))
                avg_lat = round(sum(lat)/len(lat))
                avg_lon = round(sum(lon)/len(lon))

                dic = {}
                dic['surface_temperature_celcius'] = avg_stc
                dic['confidence'] = avg_c
                dic['latitude'] = avg_lat
                dic['longitude'] = avg_lon
                
                near_hotspot.append(dic)
                
            # Check air temperature and GHI to determine cause of fire event
            flag = False
            if (climate[0]['air_temperature_celcius'] > 20 and climate[0]['GHI_w/m2'] > 180):
                flag = True

            # Now, for all hotspot data, follow date of climate data
            date = climate[0]['date']
            
            for hotspot in near_hotspot:
                hotspot['date'] = date
                time = hotspot.pop('time')
                hotspot['datetime'] = date + time
                if flag:
                    hotspot['cause'] = 'natural'
                else:
                    hotspot['cause'] = 'other'            
    
    else:
        return None
    
    client = MongoClient()
    db = client.fit3182_assignment_db
    try:
        if len(climate) != 0:
            db.climate_stream.insert(climate[0])
            cursor = db.climate_stream.find({"date" : climate[0]["date"]})
            for location in near_hotspot:
                location["foreign_key"] = cursor[0]["_id"]
                db.hotspot_stream.insert(location)
    except Exception as ex:
        print("Exception Occured. Message: {0}".format(str(ex)))
    client.close()


    
n_secs = 10
topic = 'assignment'

client = MongoClient()
db = client.fit3182_assignment_db
db.climate_stream.drop()
db.hotspot_stream.drop()

client.close()

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week11-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary

lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)